In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
dfCC = pd.read_pickle("EIPH_ClusterFilter_Annotations.pkl")
df = pd.read_pickle("EIPH_Annotations.pkl")

In [4]:
df["width"] = [vector["x2"] - vector["x1"] for vector in df['vector']]
df["height"] = [vector["y2"] - vector["y1"] for vector in df['vector']]
df["scales"] = df["width"] / df["height"]

In [5]:
df.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version,width,height,scales
0,1433998,3530,246,equine,07_EIPH_574882 Berliner Blau-001.svs,BerlinerBlau,0,"{'x1': 22691, 'x2': 22782, 'y1': 9025, 'y2': 9...",927cade0-4579-48e1-a884-a07d88594a0a,1,True,1,SDATA,Cluster,91.0,91.0,1.000000
1,1433999,3530,246,equine,07_EIPH_574882 Berliner Blau-001.svs,BerlinerBlau,0,"{'x1': 32197, 'x2': 32338, 'y1': 15975, 'y2': ...",cafc4aa3-324f-4015-9451-ad7e5bee53f3,1,False,1,SDATA,Inference,141.0,141.0,1.000000
2,1434000,3530,246,equine,07_EIPH_574882 Berliner Blau-001.svs,BerlinerBlau,0,"{'x1': 23069, 'x2': 23180, 'y1': 11514, 'y2': ...",979ef20e-6228-4cb3-ac9a-aecea6ad75c1,1,False,1,SDATA,Inference,111.0,111.0,1.000000
3,1434001,3530,246,equine,07_EIPH_574882 Berliner Blau-001.svs,BerlinerBlau,0,"{'x1': 9045, 'x2': 9148, 'y1': 9711, 'y2': 9814}",ef87d780-5e2e-413e-a2ff-0712f6aa1532,1,False,1,SDATA,Inference,103.0,103.0,1.000000
4,1434002,3530,246,equine,07_EIPH_574882 Berliner Blau-001.svs,BerlinerBlau,0,"{'x1': 10538, 'x2': 10645, 'y1': 10306, 'y2': ...",0f141dc4-5589-4e5a-a504-d77e7ad94683,1,False,1,SDATA,Inference,107.0,106.0,1.009434


In [6]:
image_data = []

for image_id, image_name in zip(df["image_id"].unique(), df["image_name"].unique()):
    
    species = df[df["image_id"] == image_id]["species"].unique()[0]
    image_type = df[df["image_id"] == image_id]["image_type"].unique()[0]
    image_set = df[df["image_id"] == image_id]["image_set"].unique()[0]
    
    image_data.append([image_id, image_name, species, image_type, image_set])
    
image_data = pd.DataFrame(image_data, columns=["image_id", "image_name", "species", "image_type", "image_set"])
image_data                      

,image_id,image_name,species,image_type,image_set
0,3530,07_EIPH_574882 Berliner Blau-001.svs,equine,BerlinerBlau,246
1,3536,08_EIPH_574999 Berliner Blau.svs,equine,BerlinerBlau,246
2,3539,09_EIPH_563417 L berliner blau.svs,equine,BerlinerBlau,246
3,3543,10_EIPH_563417 R Berliner Blau.svs,equine,BerlinerBlau,246
4,3545,13_EIPH_570370 Berliner Blau.svs,equine,BerlinerBlau,246
...,...,...,...,...,...
86,3569,25_EIPH_568150 Turnbull blue.svs,equine,TurnbullBlue,247
87,3570,26_EIPH_566482 L Turnbull blue.svs,equine,TurnbullBlue,247
88,3572,27_EIPH_571557 Turnbull blue.svs,equine,TurnbullBlue,247
89,3574,30_EIPH_568355 Turnbull blue.svs,equine,TurnbullBlue,247


### SREP

In [9]:
srep = df[df["data_set_name"]=="SREP"]
data = []

for file in srep["image_name"].unique():
    
    fileDF = srep[srep["image_name"] == file]
    
    last_total = 0
    last_score = 0
    for version in ["SREP", "SREP_V2"]:
        
        if version != "SREP":
            versionDF = fileDF[fileDF["version"] == version]
        else:
            versionDF = fileDF
            
        for image_type in fileDF["image_type"].unique():
            
            image_typeDF = versionDF[versionDF["image_type"] == image_type]
            
            total = image_typeDF.shape[0]
            score = int(image_typeDF["grade"].mean() * 100)
            _0 = image_typeDF[image_typeDF["grade"] == 0].shape[0]
            _1 = image_typeDF[image_typeDF["grade"] == 1].shape[0]
            _2 = image_typeDF[image_typeDF["grade"] == 2].shape[0]
            _3 = image_typeDF[image_typeDF["grade"] == 3].shape[0]
            _4 = image_typeDF[image_typeDF["grade"] == 4].shape[0]
            mean = image_typeDF["grade"].mean()
            std = image_typeDF["grade"].std()
            
            file = file.split("_")[0]
            if version == "SREP":
                version = version
                
                last_total = total
                last_score = score
            else:
                image_type = ""
                version = "V2"
                
                total = "{} ({:+.2f} %)".format(total, (total / last_total * 100) - 100)
                score = "{} ({:+.2f} %)".format(score, (score / last_score * 100) - 100)
                

            data.append([file, image_type, version, total, score, _0, _1, _2, _3, _4, mean, std])
            
srep = pd.DataFrame(data, columns=['file', 'image_type', 'version', 'total' ,'score', '0', '1', '2', '3', '4', 'mean', 'SD'])
srep = srep.sort_values(by=['file', 'version'])

In [10]:
# Replace  % -> \% 

srep.to_csv(header=False, index=False, float_format='%.2f').replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['01  &  BerlinerBlau  &  SREP  &  4448  &  126  &  1013  &  1782  &  1220  &  348  &  85  &  1.26  &  0.96\\',
 '01  &    &  V2  &  3724 (-16.28 %)  &  134 (+6.35 %)  &  716  &  1481  &  1112  &  339  &  76  &  1.35  &  0.96\\',
 '02  &  BerlinerBlau  &  SREP  &  12821  &  72  &  5084  &  6208  &  1453  &  65  &  11  &  0.73  &  0.68\\',
 '02  &    &  V2  &  8954 (-30.16 %)  &  81 (+12.50 %)  &  3101  &  4527  &  1257  &  58  &  11  &  0.81  &  0.70\\',
 '03  &  BerlinerBlau  &  SREP  &  6327  &  37  &  4295  &  1699  &  330  &  3  &  0  &  0.37  &  0.58\\',
 '03  &    &  V2  &  5394 (-14.75 %)  &  43 (+16.22 %)  &  3403  &  1664  &  324  &  3  &  0  &  0.43  &  0.61\\',
 '04  &  BerlinerBlau  &  SREP  &  5450  &  62  &  2552  &  2379  &  509  &  10  &  0  &  0.63  &  0.66\\',
 '04  &    &  V2  &  3439 (-36.90 %)  &  80 (+29.03 %)  &  1150  &  1822  &  458  &  9  &  0  &  0.80  &  0.66\\',
 '05  &  BerlinerBlau  &  SREP  &  2491  &  33  &  1756  &  634  &  99  &  2  &  0  &  0.34  &  

### Equine

In [11]:
def get_stat_from_df(frame):
    
    result = {}
    
    result["total"] = frame.shape[0]
    result["score"] = int(frame["grade"].mean() * 100)
    
    result["0"] = frame[frame["grade"] == 0].shape[0]
    result["1"] = frame[frame["grade"] == 1].shape[0]
    result["2"] = frame[frame["grade"] == 2].shape[0]
    result["3"] = frame[frame["grade"] == 3].shape[0]
    result["4"] = frame[frame["grade"] == 4].shape[0]
    result["mean"] = frame["grade"].mean()
    result["std"] = frame["grade"].std()
    
    return result

In [12]:
equine = df[(df["species"]=="equine") & (df["data_set_name"]!="SREP")]
data = []
data_short = []

id = 0
for file in equine["image_name"].unique():
    
    fileDF = equine[equine["image_name"] == file]
    
    last_total = 0
    last_score = 0
            
    for image_type in fileDF["image_type"].unique():
            
        file = file.split("_")[0]
        
        # add inference results
        image_typeDF = fileDF[fileDF["image_type"] == image_type]
        stats = get_stat_from_df(image_typeDF)
                
        data.append([id, file, image_type, "Inference", stats["total"], stats["score"], stats["0"]
                     , stats["1"], stats["2"], stats["3"], stats["4"], stats["mean"], stats["std"]])
        id += 1
        
        deleted = image_typeDF[(image_typeDF["deleted"] == True)]
        
        # auto cluster results
        cluster = deleted[(deleted["scales"] > 1.2) | (deleted["scales"] < 0.8) | deleted["unique_identifier"].isin(dfCC["unique_identifier"])]
        cluster_abs = image_typeDF[~image_typeDF["unique_identifier"].isin(cluster["unique_identifier"])]
        cs = get_stat_from_df(cluster_abs)
        
        total = cs["total"]
        score = cs["score"]
        total = "{} ({:+.2f} %)".format(total, (total / stats["total"] * 100) - 100)
        score = "{} ({:+.2f} %)".format(score, (score / stats["score"] * 100) - 100)
        
        data.append([id, file, "", "Cluster", total, score, cs["0"]
                     , cs["1"], cs["2"], cs["3"], cs["4"], cs["mean"], cs["std"]])
        id += 1
        
        # expert screening results
        screening = deleted[~deleted["unique_identifier"].isin(cluster["unique_identifier"])]
        screening_abs = image_typeDF[~image_typeDF["unique_identifier"].isin(screening["unique_identifier"]) 
                     & ~image_typeDF["unique_identifier"].isin(cluster["unique_identifier"])]
        s_screen = get_stat_from_df(screening_abs)
        
        total = s_screen["total"]
        score = s_screen["score"]
        total = "{} ({:+.2f} %)".format(total, (total / cs["total"] * 100) - 100)
        score = "{} ({:+.2f} %)".format(score, (score / cs["score"] * 100) - 100)
        
        data.append([id, file, "", "Screening", total, score, s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        id += 1

        # added by expert
        added = image_typeDF[(image_typeDF["deleted"] == False) & (image_typeDF["user_id"] == 12)]
        
        # classification changed by expert
        changed = image_typeDF[(image_typeDF["deleted"] == False) & (image_typeDF["last_editor"] == 12)]
            
        data.append([id, file, "", "DensityMap", total, score, s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        
        # add just the final results
        data_short.append([file, image_type, s_screen["total"], s_screen["score"], s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        
        id += 1

equine_short = pd.DataFrame(data_short, columns=['file', 'image_type', 'total' ,'score', '0', '1', '2', '3', '4', 'mean', 'SD'])
equine_short = equine_short.sort_values(by=['file'])
equine_short.head()       
            
equine = pd.DataFrame(data, columns=['id', 'file', 'image_type', 'version', 'total' ,'score', '0', '1', '2', '3', '4', 'mean', 'SD'])
equine = equine.sort_values(by=['file', "id"])
equine.head()

,id,file,image_type,version,total,score,0,1,2,3,4,mean,SD
72,72,01,TurnbullBlue,Inference,6641,143,1301,2152,2261,876,51,1.431411,0.973211
73,73,01,,Cluster,6530 (-1.67 %),144 (+0.70 %),1252,2110,2249,869,50,1.441807,0.970730
74,74,01,,Screening,4613 (-29.36 %),157 (+9.03 %),536,1577,1837,658,5,1.570561,0.877131
75,75,01,,DensityMap,4613 (-29.36 %),157 (+9.03 %),536,1577,1837,658,5,1.570561,0.877131
76,76,02,TurnbullBlue,Inference,11694,76,5205,4404,1684,394,7,0.768086,0.821910


In [13]:
equine_short.to_csv(header=False, index=False, float_format='%.2f').replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['01  &  TurnbullBlue  &  4613  &  157  &  536  &  1577  &  1837  &  658  &  5  &  1.57  &  0.88\\',
 '02  &  TurnbullBlue  &  6311  &  98  &  2094  &  2560  &  1291  &  364  &  2  &  0.99  &  0.88\\',
 '03  &  TurnbullBlue  &  5391  &  51  &  3323  &  1424  &  583  &  61  &  0  &  0.51  &  0.73\\',
 '04  &  TurnbullBlue  &  4429  &  122  &  876  &  1977  &  1270  &  305  &  1  &  1.23  &  0.84\\',
 '05  &  TurnbullBlue  &  1634  &  48  &  967  &  554  &  104  &  9  &  0  &  0.48  &  0.64\\',
 '07  &  BerlinerBlau  &  2605  &  26  &  1940  &  635  &  30  &  0  &  0  &  0.27  &  0.47\\',
 '08  &  BerlinerBlau  &  1619  &  181  &  182  &  394  &  615  &  393  &  35  &  1.82  &  0.99\\',
 '09  &  BerlinerBlau  &  1055  &  58  &  554  &  389  &  107  &  5  &  0  &  0.59  &  0.69\\',
 '10  &  TurnbullBlue  &  596  &  95  &  169  &  288  &  135  &  4  &  0  &  0.96  &  0.73\\',
 '10  &  BerlinerBlau  &  943  &  61  &  456  &  394  &  92  &  0  &  1  &  0.62  &  0.67\\',
 '11  &  TurnbullBlue

In [14]:
columns = list(equine.columns)[1:]
equine.to_csv(header=False, index=False, float_format='%.2f', columns=columns).replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['01  &  TurnbullBlue  &  Inference  &  6641  &  143  &  1301  &  2152  &  2261  &  876  &  51  &  1.43  &  0.97\\',
 '01  &    &  Cluster  &  6530 (-1.67 %)  &  144 (+0.70 %)  &  1252  &  2110  &  2249  &  869  &  50  &  1.44  &  0.97\\',
 '01  &    &  Screening  &  4613 (-29.36 %)  &  157 (+9.03 %)  &  536  &  1577  &  1837  &  658  &  5  &  1.57  &  0.88\\',
 '01  &    &  DensityMap  &  4613 (-29.36 %)  &  157 (+9.03 %)  &  536  &  1577  &  1837  &  658  &  5  &  1.57  &  0.88\\',
 '02  &  TurnbullBlue  &  Inference  &  11694  &  76  &  5205  &  4404  &  1684  &  394  &  7  &  0.77  &  0.82\\',
 '02  &    &  Cluster  &  11451 (-2.08 %)  &  77 (+1.32 %)  &  5062  &  4311  &  1679  &  393  &  6  &  0.77  &  0.82\\',
 '02  &    &  Screening  &  6311 (-44.89 %)  &  98 (+27.27 %)  &  2094  &  2560  &  1291  &  364  &  2  &  0.99  &  0.88\\',
 '02  &    &  DensityMap  &  6311 (-44.89 %)  &  98 (+27.27 %)  &  2094  &  2560  &  1291  &  364  &  2  &  0.99  &  0.88\\',
 '03  &  TurnbullBlue 

### Cat

In [15]:
cat = df[df["species"]=="felis catus"]
data = []
data_short = []

id = 0
for file in cat["image_name"].unique():
    
    fileDF = cat[cat["image_name"] == file]
    
    last_total = 0
    last_score = 0
            
    for image_type in fileDF["image_type"].unique():
            
        file = Path(file).stem.split("_")[-1]
        
        # add inference results
        image_typeDF = fileDF[fileDF["image_type"] == image_type]
        stats = get_stat_from_df(image_typeDF)
                
        data.append([id, file, image_type, "Inference", stats["total"], stats["score"], stats["0"]
                     , stats["1"], stats["2"], stats["3"], stats["4"], stats["mean"], stats["std"]])
        id += 1
        
        deleted = image_typeDF[(image_typeDF["deleted"] == True)]
        
        # auto cluster results
        cluster = deleted[(deleted["scales"] > 1.2) | (deleted["scales"] < 0.8) | deleted["unique_identifier"].isin(dfCC["unique_identifier"])]
        cluster_abs = image_typeDF[~image_typeDF["unique_identifier"].isin(cluster["unique_identifier"])]
        cs = get_stat_from_df(cluster_abs)
        
        total = cs["total"]
        score = cs["score"]
        total = "{} ({:+.2f} %)".format(total, (total / stats["total"] * 100) - 100)
        score = "{} ({:+.2f} %)".format(score, (score / stats["score"] * 100) - 100)
        
        data.append([id, file, "", "Cluster", total, score, cs["0"]
                     , cs["1"], cs["2"], cs["3"], cs["4"], cs["mean"], cs["std"]])
        id += 1
        
        # expert screening results
        screening = deleted[~deleted["unique_identifier"].isin(cluster["unique_identifier"])]
        screening_abs = image_typeDF[~image_typeDF["unique_identifier"].isin(screening["unique_identifier"]) 
                     & ~image_typeDF["unique_identifier"].isin(cluster["unique_identifier"])]
        s_screen = get_stat_from_df(screening_abs)
        
        total = s_screen["total"]
        score = s_screen["score"]
        total = "{} ({:+.2f} %)".format(total, (total / cs["total"] * 100) - 100)
        score = "{} ({:+.2f} %)".format(score, (score / cs["score"] * 100) - 100)
        
        data.append([id, file, "", "Screening", total, score, s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        id += 1

        # added by expert
        added = image_typeDF[(image_typeDF["deleted"] == False) & (image_typeDF["user_id"] == 12)]
        
        # classification changed by expert
        changed = image_typeDF[(image_typeDF["deleted"] == False) & (image_typeDF["last_editor"] == 12)]
            
        data.append([id, file, "", "DensityMap", total, score, s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        
        # add just the final results
        data_short.append([file, image_type, s_screen["total"], s_screen["score"], s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        
        id += 1

cat_short = pd.DataFrame(data_short, columns=['file', 'image_type', 'total' ,'score', '0', '1', '2', '3', '4', 'mean', 'SD'])
cat_short = cat_short.sort_values(by=['file'])
cat_short.head()       
            
cat = pd.DataFrame(data, columns=['id', 'file', 'image_type', 'version', 'total' ,'score', '0', '1', '2', '3', '4', 'mean', 'SD'])
cat = cat.sort_values(by=['file', "id"])
cat.head()

,id,file,image_type,version,total,score,0,1,2,3,4,mean,SD
0,0,1,TurnbullBlue,Inference,31631,59,12962,18585,77,4,3,0.593184,0.498086
1,1,1,,Cluster,30497 (-3.59 %),59 (+0.00 %),12432,17990,71,3,1,0.594977,0.496616
2,2,1,,Screening,30290 (-0.68 %),59 (+0.00 %),12293,17925,70,2,0,0.596600,0.495676
3,3,1,,DensityMap,30290 (-0.68 %),59 (+0.00 %),12293,17925,70,2,0,0.596600,0.495676
36,36,10,TurnbullBlue,Inference,4141,32,2821,1305,12,2,1,0.323352,0.480044


In [16]:
cat_short.to_csv(header=False, index=False, float_format='%.2f').replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['1  &  TurnbullBlue  &  30290  &  59  &  12293  &  17925  &  70  &  2  &  0  &  0.60  &  0.50\\',
 '10  &  TurnbullBlue  &  3903  &  33  &  2617  &  1272  &  12  &  2  &  0  &  0.33  &  0.48\\',
 '11  &  TurnbullBlue  &  30884  &  70  &  9304  &  21515  &  65  &  0  &  0  &  0.70  &  0.46\\',
 '12  &  TurnbullBlue  &  8756  &  32  &  5913  &  2837  &  2  &  0  &  4  &  0.33  &  0.48\\',
 '13  &  TurnbullBlue  &  17025  &  10  &  15232  &  1791  &  2  &  0  &  0  &  0.11  &  0.31\\',
 '14  &  TurnbullBlue  &  5799  &  12  &  5090  &  707  &  1  &  1  &  0  &  0.12  &  0.33\\',
 '2  &  TurnbullBlue  &  11287  &  76  &  2632  &  8636  &  19  &  0  &  0  &  0.77  &  0.43\\',
 '3  &  TurnbullBlue  &  41173  &  88  &  5433  &  35246  &  478  &  15  &  1  &  0.88  &  0.36\\',
 '4  &  TurnbullBlue  &  38063  &  68  &  11970  &  25980  &  100  &  5  &  8  &  0.69  &  0.47\\',
 '5  &  TurnbullBlue  &  35696  &  56  &  15583  &  20110  &  2  &  1  &  0  &  0.56  &  0.50\\',
 '6  &  TurnbullBlue 

In [17]:
columns = list(equine.columns)[1:]
cat.to_csv(header=False, index=False, float_format='%.2f', columns=columns).replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['1  &  TurnbullBlue  &  Inference  &  31631  &  59  &  12962  &  18585  &  77  &  4  &  3  &  0.59  &  0.50\\',
 '1  &    &  Cluster  &  30497 (-3.59 %)  &  59 (+0.00 %)  &  12432  &  17990  &  71  &  3  &  1  &  0.59  &  0.50\\',
 '1  &    &  Screening  &  30290 (-0.68 %)  &  59 (+0.00 %)  &  12293  &  17925  &  70  &  2  &  0  &  0.60  &  0.50\\',
 '1  &    &  DensityMap  &  30290 (-0.68 %)  &  59 (+0.00 %)  &  12293  &  17925  &  70  &  2  &  0  &  0.60  &  0.50\\',
 '10  &  TurnbullBlue  &  Inference  &  4141  &  32  &  2821  &  1305  &  12  &  2  &  1  &  0.32  &  0.48\\',
 '10  &    &  Cluster  &  3903 (-5.75 %)  &  33 (+3.12 %)  &  2617  &  1272  &  12  &  2  &  0  &  0.33  &  0.48\\',
 '10  &    &  Screening  &  3903 (+0.00 %)  &  33 (+0.00 %)  &  2617  &  1272  &  12  &  2  &  0  &  0.33  &  0.48\\',
 '10  &    &  DensityMap  &  3903 (+0.00 %)  &  33 (+0.00 %)  &  2617  &  1272  &  12  &  2  &  0  &  0.33  &  0.48\\',
 '11  &  TurnbullBlue  &  Inference  &  31972  &  69  &  9

### Human

In [18]:
human = df[df["species"]=="human"]
data = []
data_short = []

id = 0
for file in human["image_name"].unique():
    
    fileDF = human[human["image_name"] == file]
    
    last_total = 0
    last_score = 0
            
    for image_type in fileDF["image_type"].unique():
            
        file = Path(file).stem.split("_")[0]
        
        # add inference results
        image_typeDF = fileDF[fileDF["image_type"] == image_type]
        stats = get_stat_from_df(image_typeDF)
                
        data.append([id, file, image_type, "Inference", stats["total"], stats["score"], stats["0"]
                     , stats["1"], stats["2"], stats["3"], stats["4"], stats["mean"], stats["std"]])
        id += 1
        
        deleted = image_typeDF[(image_typeDF["deleted"] == True)]
        
        # auto cluster results
        cluster = deleted[(deleted["scales"] > 1.2) | (deleted["scales"] < 0.8) | deleted["unique_identifier"].isin(dfCC["unique_identifier"])]
        cluster_abs = image_typeDF[~image_typeDF["unique_identifier"].isin(cluster["unique_identifier"])]
        cs = get_stat_from_df(cluster_abs)
        
        total = cs["total"]
        score = cs["score"]
        total = "{} ({:+.2f} %)".format(total, (total / stats["total"] * 100) - 100)
        score = "{} ({:+.2f} %)".format(score, (score / stats["score"] * 100) - 100)
        
        data.append([id, file, "", "Cluster", total, score, cs["0"]
                     , cs["1"], cs["2"], cs["3"], cs["4"], cs["mean"], cs["std"]])
        id += 1
        
        # expert screening results
        screening = deleted[~deleted["unique_identifier"].isin(cluster["unique_identifier"])]
        screening_abs = image_typeDF[~image_typeDF["unique_identifier"].isin(screening["unique_identifier"]) 
                     & ~image_typeDF["unique_identifier"].isin(cluster["unique_identifier"])]
        s_screen = get_stat_from_df(screening_abs)
        
        total = s_screen["total"]
        score = s_screen["score"]
        total = "{} ({:+.2f} %)".format(total, (total / cs["total"] * 100) - 100)
        score = "{} ({:+.2f} %)".format(score, (score / cs["score"] * 100) - 100)
        
        data.append([id, file, "", "Screening", total, score, s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        id += 1

        # added by expert
        added = image_typeDF[(image_typeDF["deleted"] == False) & (image_typeDF["user_id"] == 12)]
        
        # classification changed by expert
        changed = image_typeDF[(image_typeDF["deleted"] == False) & (image_typeDF["last_editor"] == 12)]
            
        data.append([id, file, "", "DensityMap", total, score, s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        
        # add just the final results
        data_short.append([file, image_type, s_screen["total"], s_screen["score"], s_screen["0"]
                     , s_screen["1"], s_screen["2"], s_screen["3"], s_screen["4"], s_screen["mean"], s_screen["std"]])
        
        id += 1

human_short = pd.DataFrame(data_short, columns=['file', 'image_type', 'total' ,'score', '0', '1', '2', '3', '4', 'mean', 'SD'])
human_short = human_short.sort_values(by=['file'])
human_short.head()       
            
human = pd.DataFrame(data, columns=['id', 'file', 'image_type', 'version', 'total' ,'score', '0', '1', '2', '3', '4', 'mean', 'SD'])
human = human.sort_values(by=['file', "id"])
human.head()

,id,file,image_type,version,total,score,0,1,2,3,4,mean,SD
8,8,10052,BerlinerBlau,Inference,16088,177,1057,3792,8926,2284,29,1.778468,0.773724
9,9,10052,,Cluster,16083 (-0.03 %),177 (+0.00 %),1057,3791,8923,2283,29,1.778400,0.773754
10,10,10052,,Screening,6347 (-60.54 %),180 (+1.69 %),120,1736,3744,738,9,1.807783,0.658946
11,11,10052,,DensityMap,6347 (-60.54 %),180 (+1.69 %),120,1736,3744,738,9,1.807783,0.658946
12,12,10080,BerlinerBlau,Inference,11216,131,1431,5386,3799,582,18,1.319722,0.765593


In [19]:
human_short.to_csv(header=False, index=False, float_format='%.2f').replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['10052  &  BerlinerBlau  &  6347  &  180  &  120  &  1736  &  3744  &  738  &  9  &  1.81  &  0.66\\',
 '10080  &  BerlinerBlau  &  6025  &  146  &  284  &  3015  &  2375  &  342  &  9  &  1.47  &  0.68\\',
 '10080  &  TurnbullBlue  &  13833  &  135  &  1185  &  7135  &  4879  &  620  &  14  &  1.36  &  0.71\\',
 '10120  &  BerlinerBlau  &  8600  &  191  &  112  &  2225  &  4656  &  1502  &  105  &  1.91  &  0.73\\',
 '10120  &  TurnbullBlue  &  17143  &  192  &  387  &  3602  &  10171  &  2947  &  36  &  1.92  &  0.69\\',
 '10227  &  BerlinerBlau  &  24817  &  65  &  9663  &  13975  &  1161  &  16  &  2  &  0.66  &  0.57\\',
 '10277  &  TurnbullBlue  &  25240  &  70  &  9500  &  13797  &  1910  &  32  &  1  &  0.70  &  0.61\\',
 '10672  &  BerlinerBlau  &  2651  &  216  &  261  &  271  &  1000  &  1009  &  110  &  2.16  &  1.01\\',
 '10672  &  TurnbullBlue  &  2094  &  211  &  154  &  270  &  940  &  648  &  82  &  2.11  &  0.94\\',
 '10835  &  BerlinerBlau  &  18430  &  134  &  596 

In [20]:
columns = list(equine.columns)[1:]
human.to_csv(header=False, index=False, float_format='%.2f', columns=columns).replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['10052  &  BerlinerBlau  &  Inference  &  16088  &  177  &  1057  &  3792  &  8926  &  2284  &  29  &  1.78  &  0.77\\',
 '10052  &    &  Cluster  &  16083 (-0.03 %)  &  177 (+0.00 %)  &  1057  &  3791  &  8923  &  2283  &  29  &  1.78  &  0.77\\',
 '10052  &    &  Screening  &  6347 (-60.54 %)  &  180 (+1.69 %)  &  120  &  1736  &  3744  &  738  &  9  &  1.81  &  0.66\\',
 '10052  &    &  DensityMap  &  6347 (-60.54 %)  &  180 (+1.69 %)  &  120  &  1736  &  3744  &  738  &  9  &  1.81  &  0.66\\',
 '10080  &  BerlinerBlau  &  Inference  &  11216  &  131  &  1431  &  5386  &  3799  &  582  &  18  &  1.32  &  0.77\\',
 '10080  &    &  Cluster  &  11216 (+0.00 %)  &  131 (+0.00 %)  &  1431  &  5386  &  3799  &  582  &  18  &  1.32  &  0.77\\',
 '10080  &    &  Screening  &  6025 (-46.28 %)  &  146 (+11.45 %)  &  284  &  3015  &  2375  &  342  &  9  &  1.47  &  0.68\\',
 '10080  &    &  DensityMap  &  6025 (-46.28 %)  &  146 (+11.45 %)  &  284  &  3015  &  2375  &  342  &  9  &  1.47  & 

### summary

In [26]:
temp_human, temp_cat, temp_equine = human, cat, equine
temp_human["species"] = "human"
temp_cat["species"] = "felis catus"
temp_equine["species"] = "equine"

totalDf = pd.concat([temp_human, temp_cat, temp_equine])
totalDf.head()

,id,file,image_type,version,total,score,0,1,2,3,4,mean,SD,species,temp_equine
8,8,10052,BerlinerBlau,Inference,16088,177,1057,3792,8926,2284,29,1.778468,0.773724,human,NaN
9,9,10052,,Cluster,16083 (-0.03 %),177 (+0.00 %),1057,3791,8923,2283,29,1.778400,0.773754,human,NaN
10,10,10052,,Screening,6347 (-60.54 %),180 (+1.69 %),120,1736,3744,738,9,1.807783,0.658946,human,NaN
11,11,10052,,DensityMap,6347 (-60.54 %),180 (+1.69 %),120,1736,3744,738,9,1.807783,0.658946,human,NaN
12,12,10080,BerlinerBlau,Inference,11216,131,1431,5386,3799,582,18,1.319722,0.765593,human,NaN


In [53]:
srep.head()

,file,image_type,version,total,score,0,1,2,3,4,mean,SD
28,01,BerlinerBlau,SREP,4448,126,1013,1782,1220,348,85,1.260342,0.958848
29,01,,V2,3724 (-16.28 %),134 (+6.35 %),716,1481,1112,339,76,1.349624,0.957433
18,02,BerlinerBlau,SREP,12821,72,5084,6208,1453,65,11,0.729506,0.681716
19,02,,V2,8954 (-30.16 %),81 (+12.50 %),3101,4527,1257,58,11,0.810699,0.698496
30,03,BerlinerBlau,SREP,6327,37,4295,1699,330,3,0,0.374269,0.584299


In [58]:
data = []
total_slides = []

id = 0
for species in totalDf["species"].unique():
    
    speciesDf = totalDf[totalDf["species"] == species]
    total_slides.append(len(speciesDf))
    
    last_total = 0
    last_score = 0
    for version in ["Inference", "Cluster", "Screening", "DensityMap"]:
    
        versionDf = speciesDf[speciesDf["version"] == version]
        
        num_slides = len(versionDf)
        total = sum(versionDf[["0", "1", "2", "3", "4"]].sum())
        _0, _1, _2, _3, _4 = versionDf[["0", "1", "2", "3", "4"]].sum()
        
        cell_array = np.hstack([np.ones(shape=(_0)) * 0, np.ones(shape=(_1)) * 1, np.ones(shape=(_2)) * 2, np.ones(shape=(_3)) * 3, np.ones(shape=(_4)) * 4])
        score = float(cell_array.mean() * 100)
        mean = cell_array.mean()
        SD = cell_array.std()
        
        if version != "Inference":
            total = "{} ({:+.2f} %)".format(total, (total / last_total * 100) - 100)
            score = "{} ({:+.2f} %)".format(score, (score / last_score * 100) - 100)
        
        last_total = sum(versionDf[["0", "1", "2", "3", "4"]].sum())
        last_score = float(cell_array.mean() * 100)
        
        
        data.append([species, "SDATA" , num_slides, version, total, score, _0, _1, _2, _3, _4, mean, SD])
        
        id += 1
        
        
last_total = 0
last_score = 0
total_slides.append(len(srep))
for version in ["SREP", "V2"]:
    
    versionDf = srep[srep["version"] == version] 
        
    num_slides = len(versionDf)
    total = sum(versionDf[["0", "1", "2", "3", "4"]].sum())
    _0, _1, _2, _3, _4 = versionDf[["0", "1", "2", "3", "4"]].sum()
        
    cell_array = np.hstack([np.ones(shape=(_0)) * 0, np.ones(shape=(_1)) * 1, np.ones(shape=(_2)) * 2, np.ones(shape=(_3)) * 3, np.ones(shape=(_4)) * 4])
    score = float(cell_array.mean() * 100)
    mean = cell_array.mean()
    SD = cell_array.std()

    if version != "SREP":
        version = "Screening"
        total = "{} ({:+.2f} %)".format(total, (total / last_total * 100) - 100)
        score = "{} ({:+.2f} %)".format(score, (score / last_score * 100) - 100)
        
    last_total = sum(versionDf[["0", "1", "2", "3", "4"]].sum())
    last_score = float(cell_array.mean() * 100)
        
        
    data.append([species, "SREP", num_slides, version, total, score, _0, _1, _2, _3, _4, mean, SD])
        
    id += 1

        
speciesStatDf = pd.DataFrame(data, columns=["species", "dataset" ,"slides", "version", "total", "score", "0", "1", "2", "3", "4", "mean", "SD"])
speciesStatDf

,species,dataset,slides,version,total,score,0,1,2,3,4,mean,SD
0,human,SDATA,20,Inference,376323,131.394,66712,158427,120438,27822,2924,1.313938,0.874305
1,human,SDATA,20,Cluster,376317 (-0.00 %),131.39268223332988 (-0.00 %),66712,158426,120434,27821,2924,1.313927,0.874305
2,human,SDATA,20,Screening,351127 (-6.69 %),128.58139647477978 (-2.14 %),64200,152154,107779,24204,2790,1.285814,0.869105
3,human,SDATA,20,DensityMap,351127 (+0.00 %),128.58139647477978 (+0.00 %),64200,152154,107779,24204,2790,1.285814,0.869105
4,felis catus,SDATA,14,Inference,309039,57.8044,135199,170460,2464,414,502,0.578044,0.536084
5,felis catus,SDATA,14,Cluster,297445 (-3.75 %),57.72092319588496 (-0.14 %),129034,165619,2350,399,43,0.577209,0.519253
6,felis catus,SDATA,14,Screening,297238 (-0.07 %),57.73622484339149 (+0.03 %),128895,165554,2349,398,42,0.577362,0.519182
7,felis catus,SDATA,14,DensityMap,297238 (+0.00 %),57.73622484339149 (+0.00 %),128895,165554,2349,398,42,0.577362,0.519182
8,equine,SDATA,40,Inference,255074,100.464,98519,82062,51123,21531,1839,1.004638,0.994570
9,equine,SDATA,40,Cluster,250167 (-1.92 %),100.94616795980285 (+0.48 %),95993,80564,50663,21144,1803,1.009462,0.994528


In [59]:
data = []

    
last_total = 0
last_score = 0
for version in ["Inference", "Cluster", "Screening", "DensityMap"]:
    
    versionDf = speciesStatDf[speciesStatDf["version"] == version]
    total = sum(versionDf[["0", "1", "2", "3", "4"]].sum())
    _0, _1, _2, _3, _4 = versionDf[["0", "1", "2", "3", "4"]].sum()
        
    cell_array = np.hstack([np.ones(shape=(_0)) * 0, np.ones(shape=(_1)) * 1, np.ones(shape=(_2)) * 2, np.ones(shape=(_3)) * 3, np.ones(shape=(_4)) * 4])
    score = float(cell_array.mean() * 100)
    mean = cell_array.mean()
    SD = cell_array.std()
    
    if version != "Inference":
        total = "{} ({:+.2f} %)".format(total, (total / last_total * 100) - 100)
        score = "{} ({:+.2f} %)".format(score, (score / last_score * 100) - 100)
        
    last_total = sum(versionDf[["0", "1", "2", "3", "4"]].sum())
    last_score = float(cell_array.mean() * 100)
        
        
    data.append(["Total", "SDATA" , len(total_slides), version, total, score, _0, _1, _2, _3, _4, mean, SD])
    
tempDf = pd.DataFrame(data, columns=["species", "dataset" ,"slides", "version", "total", "score", "0", "1", "2", "3", "4", "mean", "SD"])

tempDf = pd.concat([speciesStatDf, tempDf])
tempDf.head()

,species,dataset,slides,version,total,score,0,1,2,3,4,mean,SD
0,human,SDATA,20,Inference,376323,131.394,66712,158427,120438,27822,2924,1.313938,0.874305
1,human,SDATA,20,Cluster,376317 (-0.00 %),131.39268223332988 (-0.00 %),66712,158426,120434,27821,2924,1.313927,0.874305
2,human,SDATA,20,Screening,351127 (-6.69 %),128.58139647477978 (-2.14 %),64200,152154,107779,24204,2790,1.285814,0.869105
3,human,SDATA,20,DensityMap,351127 (+0.00 %),128.58139647477978 (+0.00 %),64200,152154,107779,24204,2790,1.285814,0.869105
4,felis catus,SDATA,14,Inference,309039,57.8044,135199,170460,2464,414,502,0.578044,0.536084


In [60]:
columns = list(equine.columns)[1:]
tempDf.to_csv(header=False, index=False, float_format='%.2f').replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['human  &  SDATA  &  20  &  Inference  &  376323  &  131.39377609128329  &  66712  &  158427  &  120438  &  27822  &  2924  &  1.31  &  0.87\\',
 'human  &  SDATA  &  20  &  Cluster  &  376317 (-0.00 %)  &  131.39268223332988 (-0.00 %)  &  66712  &  158426  &  120434  &  27821  &  2924  &  1.31  &  0.87\\',
 'human  &  SDATA  &  20  &  Screening  &  351127 (-6.69 %)  &  128.58139647477978 (-2.14 %)  &  64200  &  152154  &  107779  &  24204  &  2790  &  1.29  &  0.87\\',
 'human  &  SDATA  &  20  &  DensityMap  &  351127 (+0.00 %)  &  128.58139647477978 (+0.00 %)  &  64200  &  152154  &  107779  &  24204  &  2790  &  1.29  &  0.87\\',
 'felis catus  &  SDATA  &  14  &  Inference  &  309039  &  57.80435479017212  &  135199  &  170460  &  2464  &  414  &  502  &  0.58  &  0.54\\',
 'felis catus  &  SDATA  &  14  &  Cluster  &  297445 (-3.75 %)  &  57.72092319588496 (-0.14 %)  &  129034  &  165619  &  2350  &  399  &  43  &  0.58  &  0.52\\',
 'felis catus  &  SDATA  &  14  &  Screening  